GLCM feature performed dataset for dear kagglers 

Thank you hklee's kernal " https://www.kaggle.com/zeemeen/glcm-texture-features 

In [ ]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

import time
t_start = time.time()

In [ ]:
version = 5
basic_name = f'Unet_resnet_v{version}'
save_model_name = basic_name + '.model'
submission_file = basic_name + '.csv'

print(save_model_name)
print(submission_file)

In [ ]:
img_size_ori = 101
img_size_target = 101

def upsample(img):# not used
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    
def downsample(img):# not used
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

In [ ]:
# Loading of training/testing ids and depths
train_df = pd.read_csv("../input/glcm-tgs/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/glcm-tgs/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)

In [ ]:
train_df["images"] = [np.array(load_img("../input/glcm-tgs/train/images/{}.png".format(idx), color_mode='rgb')) / 765 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("../input/glcm-tgs/train/mask/{}.png".format(idx), color_mode='rgb')) / 765 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
#Plotting the depth distributions¶

sns.distplot(train_df.z, label="Train")
sns.distplot(test_df.z, label="Test")
plt.legend()
plt.title("Depth distribution")

In [ ]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-3, img_size_target, img_size_target, 3), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-3, img_size_target, img_size_target, 3), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state=555)

In [ ]:
tmp_img1 = np.zeros((img_size_target, img_size_target,3), dtype=train_df.images.loc[ids_train[100]].dtype)
tmp_img1[:img_size_ori, :img_size_ori] = train_df.images.loc[ids_train[100]]
fix, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].imshow(tmp_img1, cmap="Greys")
axs[0].set_title("Original image")
axs[1].imshow(x_train[100].squeeze(), cmap="Greys")
axs[1].set_title("Scaled image")

In [ ]:
tmp_img2 = np.zeros((img_size_target, img_size_target,3), dtype=train_df.masks.loc[ids_train[100]].dtype)
tmp_img2[:img_size_ori, :img_size_ori] = train_df.masks.loc[ids_train[100]]
fix, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].imshow(tmp_img2, cmap="Greys")
axs[0].set_title("Original image_mask")
axs[1].imshow(y_train[100].squeeze(), cmap="Greys")
axs[1].set_title("Scaled image")